In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
from songs import *
import copy
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))
    
def NextBatch():
    batch = np.empty((64, 13*128))
    y = []
    for i in range(64):
        song = copy.deepcopy(Songs[np.random.randint(0, len(Songs))])
        y.append([np.random.randint(0, 9) * np.random.randint(0, 9)])
        song.noise(y[-1][0])
        batch[i] = song.notes.flatten()
    return batch, (1 - np.array(y) / 64)

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [4]:
'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Training Parameters
learning_rate = 0.001
training_steps = 5000
batch_size = 64
display_step = 200

# Network Parameters
num_input = 13 # MNIST data input (img shape: 28*28)
timesteps = 128 # timesteps
num_hidden = 100 # hidden layer num of features
num_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

#logits = RNN(X, weights, biases)
outputs = RNN(X, weights, biases)

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
loss_op = tf.reduce_mean(tf.squared_difference(outputs, Y))
#train_op = optimizer.minimize(loss_op)
train_op = tf.contrib.layers.optimize_loss(
    loss_op, tf.contrib.framework.get_global_step(), optimizer=optimizer,
    learning_rate=learning_rate)

# Evaluate model (with test logits, for dropout to be disabled)
#correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

for step in range(1, training_steps+1):
    batch_x, batch_y = NextBatch()#mnist.train.next_batch(batch_size)
    # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        L = sess.run([loss_op], feed_dict={X: batch_x, Y: batch_y})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L[0]))

print("Optimization Finished!")

Step 1, Minibatch Loss= 1.5116
Step 200, Minibatch Loss= 0.0512
Step 400, Minibatch Loss= 0.0487
Step 600, Minibatch Loss= 0.0465
Step 800, Minibatch Loss= 0.0420
Step 1000, Minibatch Loss= 0.0585
Step 1200, Minibatch Loss= 0.0455
Step 1400, Minibatch Loss= 0.0599
Step 1600, Minibatch Loss= 0.0321
Step 1800, Minibatch Loss= 0.0490
Step 2000, Minibatch Loss= 0.0503
Step 2200, Minibatch Loss= 0.0545
Step 2400, Minibatch Loss= 0.0390
Step 2600, Minibatch Loss= 0.0485
Step 2800, Minibatch Loss= 0.0291
Step 3000, Minibatch Loss= 0.0506
Step 3200, Minibatch Loss= 0.0238
Step 3400, Minibatch Loss= 0.0338
Step 3600, Minibatch Loss= 0.0369
Step 3800, Minibatch Loss= 0.0301
Step 4000, Minibatch Loss= 0.0384
Step 4200, Minibatch Loss= 0.0397
Step 4400, Minibatch Loss= 0.0471
Step 4600, Minibatch Loss= 0.0416
Step 4800, Minibatch Loss= 0.0503
Step 5000, Minibatch Loss= 0.0459
Step 5200, Minibatch Loss= 0.0332
Step 5400, Minibatch Loss= 0.0323
Step 5600, Minibatch Loss= 0.0320
Step 5800, Minibatch 

In [5]:
batch_x, batch_y = NextBatch()
batch_x = batch_x.reshape((batch_size, timesteps, num_input))

In [6]:
outp = sess.run([outputs], feed_dict={X: batch_x, Y: batch_y})

In [7]:
outp = np.array(outp)[0, :, 0]
batch_y = batch_y[:, 0]

In [8]:
outp[batch_y == 1]

array([ 0.97578901,  0.7414425 ,  0.81117874,  0.90079254,  0.9898544 ,
        0.85046381,  1.01881909,  0.87322015,  0.81117874,  0.81771642,
        0.89996713,  0.86539155,  0.91882735,  0.80229038,  0.93399626], dtype=float32)

In [9]:
outp[batch_y < 0.5]

array([ 0.33812016,  0.71158487,  0.34896761,  0.24188811,  0.45424467,
        0.41433007,  0.52510244,  0.48307043,  0.51990777,  0.72696286], dtype=float32)

In [21]:
var_grad = tf.gradients(outputs, [X])

In [48]:
noise = np.random.uniform(0, 1, (1, 128, 13))

In [49]:
a = 1
for i in range(100):
    vg, rew = sess.run([var_grad, outputs], feed_dict={X: noise})
    noise += np.array(a * vg[0])
    print(rew)
    a *= 0.999

[[-5.84583712]]
[[-4.99764395]]
[[-4.08316994]]
[[-3.11377263]]
[[-2.11158085]]
[[-1.10752583]]
[[-0.13474917]]
[[ 0.77954394]]
[[ 1.6190567]]
[[ 2.37832475]]
[[ 3.05952477]]
[[ 3.66892028]]
[[ 4.21432972]]
[[ 4.70371056]]
[[ 5.14451694]]
[[ 5.54344749]]
[[ 5.906394]]
[[ 6.23847723]]
[[ 6.54411316]]
[[ 6.82707024]]
[[ 7.09056568]]
[[ 7.33732414]]
[[ 7.56964207]]
[[ 7.78946686]]
[[ 7.99841595]]
[[ 8.19787407]]
[[ 8.38898277]]
[[ 8.57270813]]
[[ 8.74986076]]
[[ 8.92111778]]
[[ 9.08704758]]
[[ 9.24812603]]
[[ 9.40476704]]
[[ 9.55730247]]
[[ 9.70602703]]
[[ 9.85118961]]
[[ 9.99300575]]
[[ 10.13166142]]
[[ 10.26732063]]
[[ 10.40013409]]
[[ 10.53022766]]
[[ 10.6577282]]
[[ 10.78273869]]
[[ 10.90535927]]
[[ 11.02568531]]
[[ 11.14380455]]
[[ 11.25979996]]
[[ 11.37374115]]
[[ 11.48570824]]
[[ 11.59576797]]
[[ 11.70398235]]
[[ 11.81041336]]
[[ 11.91512871]]
[[ 12.0181694]]
[[ 12.11959743]]
[[ 12.21946335]]
[[ 12.31781578]]
[[ 12.41469288]]
[[ 12.51014423]]
[[ 12.60420799]]
[[ 12.69692612]]
[[ 12

In [75]:
test = (noise > 0.9)[0]
test.sum()

135

In [78]:
#MySong(test).save_file("LSTM listener output maximized")

## а ещё йауметьвгенетику

In [128]:
population = np.random.choice([0, 1], (1000, 128, 13), p=[0.9, 0.1])

In [129]:
for gen in range(25):
    rewards = sess.run([outputs], feed_dict={X: population})[0]

    i = np.argsort(rewards.flatten())[-250:]
    print(rewards[i[0]], rewards[i[-1]])
    population = population[i]

    mothers = np.random.randint(0, 250, 1000)
    fathers = np.random.randint(0, 250, 1000)
    children = (population[mothers] + population[fathers]) / 2
    children[children == 0.5] = np.random.choice([0, 1], ((children == 0.5).sum()), p=[0.5, 0.5])

    population = children

[-0.34529352] [ 0.3968932]
[-0.01950306] [ 0.46789747]
[ 0.2118029] [ 0.58066291]
[ 0.40004498] [ 0.69723481]
[ 0.55638939] [ 0.78155404]
[ 0.6818009] [ 0.86992091]
[ 0.77621418] [ 0.94908482]
[ 0.8563686] [ 0.993213]
[ 0.92054552] [ 1.01440358]
[ 0.97190648] [ 1.06331015]
[ 1.01419592] [ 1.07445812]
[ 1.04561257] [ 1.10207033]
[ 1.07110286] [ 1.11727452]
[ 1.09249878] [ 1.12866402]
[ 1.11143422] [ 1.13712382]
[ 1.1242702] [ 1.14243388]
[ 1.13467026] [ 1.15202308]
[ 1.14299679] [ 1.15806055]
[ 1.14917779] [ 1.1593225]
[ 1.15447164] [ 1.16132617]
[ 1.15858865] [ 1.16584468]
[ 1.16175127] [ 1.1665318]
[ 1.16426682] [ 1.16876602]
[ 1.16628718] [ 1.16898417]
[ 1.16784787] [ 1.1706264]


### :(

In [134]:
population[-1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0., 

In [135]:
MySong(population[-1]).play()